# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 11, 3, 10, 0

while True:
    dt = datetime.now()
    if dt.month >= month:
        if dt.day > day or dt.month > month:
            break
        if dt.day == day and (dt.hour > hour or (dt.hour == hour and dt.minute > minute)):
            break
    sleep(60)

# Parameter optimization

In [2]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
opt_limit = 300
load = False

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'RSI': {'timeperiod': [12, 14, 16], 'low_bound': [15, 20, 25, 30, 35]},
              'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                        'slowd_period': [3], 'low_bound': [10, 15, 20, 25, 30]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Number of combinations is 75


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [06:01<00:00,  4.81s/it]


# Check local statistics

In [3]:
stat = pd.read_pickle('opt_STOCH_RSI_sell_5m_1h.pkl')
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat[stat['STOCH_low_bound'] == 15].sort_values('forecast_rank', ascending=False).head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank
26,STOCH_RSI,14,15,9,7,3,15,92.86,64.29,64.29,64.29,57.14,57.14,57.14,57.14,64.29,64.29,64.29,71.43,57.14,64.29,64.29,64.29,57.14,64.29,50.00,64.29,57.14,50.00,57.14,57.14,-0.70,-0.42,-0.42,-0.59,-0.45,-0.51,-0.48,-0.26,-0.28,-0.83,-0.90,-0.87,-0.87,-0.65,-0.53,-0.52,-0.43,-0.47,-0.31,-0.36,-0.25,0.06,-0.40,-0.12,14,61.905833,-0.481667,-113.318333,-6.743333
51,STOCH_RSI,16,15,9,7,3,15,92.31,61.54,61.54,61.54,53.85,53.85,53.85,53.85,61.54,61.54,61.54,69.23,53.85,61.54,61.54,61.54,53.85,61.54,46.15,61.54,53.85,46.15,53.85,53.85,-0.75,-0.47,-0.45,-0.38,-0.56,-0.41,-0.62,-0.31,-0.31,-0.81,-0.93,-0.84,-0.87,-0.66,-0.56,-0.55,-0.25,-0.43,0.00,-0.25,-0.43,0.31,-0.25,-0.19,13,58.976250,-0.457083,-143.308750,-5.942083
1,STOCH_RSI,12,15,9,7,3,15,89.47,68.42,68.42,68.42,63.16,57.89,57.89,57.89,63.16,52.63,52.63,57.89,47.37,52.63,47.37,47.37,42.11,47.37,36.84,47.37,42.11,36.84,42.11,42.11,-0.61,-0.36,-0.40,-0.32,-0.32,-0.32,-0.16,-0.16,-0.16,-0.07,-0.02,-0.41,0.27,-0.36,0.16,0.07,0.27,0.15,0.14,0.41,0.39,0.80,0.95,0.61,19,53.727917,0.022917,-309.169583,0.435417
56,STOCH_RSI,16,20,9,7,3,15,88.46,65.38,61.54,57.69,57.69,61.54,61.54,61.54,57.69,50.00,50.00,50.00,42.31,46.15,42.31,50.00,42.31,46.15,38.46,46.15,42.31,38.46,42.31,42.31,-0.61,-0.33,-0.29,-0.29,-0.23,-0.35,-0.25,-0.14,-0.14,-0.03,-0.01,-0.12,0.14,0.09,0.08,0.01,0.21,0.20,0.18,0.27,0.31,0.63,0.78,0.67,26,51.762500,0.032500,-474.175000,0.845000
31,STOCH_RSI,14,20,9,7,3,15,82.50,67.50,60.00,57.50,55.00,60.00,55.00,50.00,50.00,45.00,42.50,42.50,37.50,42.50,40.00,42.50,35.00,37.50,32.50,37.50,35.00,32.50,35.00,37.50,-0.26,-0.16,-0.11,-0.24,-0.12,-0.22,-0.12,-0.04,-0.04,0.07,0.04,0.00,0.29,0.22,0.28,0.23,0.37,0.59,0.73,0.65,0.79,0.85,0.97,1.00,40,46.354167,0.240417,-945.833333,9.616667
6,STOCH_RSI,12,20,9,7,3,15,76.19,61.90,58.73,53.97,50.79,57.14,49.21,47.62,44.44,38.10,33.33,31.75,30.16,33.33,31.75,31.75,26.98,28.57,25.40,30.16,28.57,26.98,28.57,30.16,-0.20,-0.14,-0.08,-0.12,-0.07,-0.10,0.00,0.00,0.00,0.17,0.25,0.33,0.38,0.41,0.40,0.45,0.45,0.63,0.77,0.80,0.95,0.95,1.01,1.03,63,39.814583,0.344583,-1901.681250,21.708750
61,STOCH_RSI,16,25,9,7,3,15,79.22,58.44,55.84,53.25,50.65,55.84,46.75,44.16,42.86,35.06,32.47,31.17,28.57,31.17,29.87,27.27,23.38,24.68,22.08,25.97,24.68,23.38,22.08,24.68,-0.20,-0.14,-0.08,-0.10,-0.04,-0.10,0.00,0.00,0.08,0.17,0.32,0.43,0.50,0.49,0.44,0.75,0.71,0.84,0.86,0.95,1.00,1.11,1.11,1.21,77,37.230000,0.429583,-2523.290000,33.077917
36,STOCH_RSI,14,25,9,7,3,15,74.76,58.25,55.34,50.49,46.60,54.37,45.63,42.72,37.86,32.04,30.10,29.13,28.16,30.10,29.13,27.18,22.33,23.30,21.36,24.27,23.30,22.33,21.36,23.30,-0.18,-0.09,-0.07,-0.06,0.00,-0.08,0.00,0.08,0.14,0.24,0.32,0.41,0.50,0.52,0.50,0.70,0.71,0.81,0.81,0.81,0.95,0.95,1.01,1.03,103,35.558750,0.417083,-3547.448750,42.959583
11,STOCH_RSI,12,25,9,7,3,15,71.54,54.62,47.69,46.15,43.85,49.23,40.00,36.92,34.62

# Check global statistics

In [3]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            9                  7                  3                  30                            83.541667   
                                                                                      25                            85.785833   
12             25            9                  7                  3                  30                            88.782083   
14             30            9                  7                  3                  25                            80.833333   
12             25            9                  7                  3                  25                            87.847917   
14             25            9                  7                  3                  30                            89.395000   
                                                                                      20                            89.395000   
                                                                                      25                            89.395000   
12             25            9                  7                  3                  20                            89.395000   
16             30            9                  7                  3                  20                            84.820833   
14             30            9                  7                  3                  30                            78.804583   
16             25            9                  7                  3                  20                            88.750000   
                                                                                      25                            88.750000   
                                                                                      30                            88.750000   
12             30            9                  7                  3                  30                            75.416667   
14             30            9                  7                  3                  20                            79.687500   
16             15            5                  4                  3                  25                            93.229167   
12             25            9                  7                  3                  15                            91.664583   
14             25            9                  7                  3                  15                            91.664583   
16             15            7                  4                  5                  25                            95.025694   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            9                  7                  3                  30                        -1.760000   
                                                                                      25                        -1.970833   
12             25            9                  7                  3                  30                        -2.084167   
14             30            9                  7                  3                  25                        -1.777917   
12             25            9                  7                  3                  25                        -2.421250   
14             25            9                  7                  3                  30                        -2.759583   
                                                                                      20                        -2.759583   
                                                                                      25                        -2.759583   
12            

# Save new config data to config file

In [4]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'RSI': {'timeperiod': [14], 'low_bound': [25]},
                  'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                            'slowd_period': [3], 'low_bound': [15]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

In [2]:
total_stat.loc[(14, 35, 9, 3, 7, 25)]

pct_right_forecast_avg      59.748902
pct_price_diff_avg          -0.362008
forecast_rank            -1112.210871
price_rank                 -43.262538
forecasts_num             1174.000000
Name: (14, 35, 9, 3, 7, 25), dtype: float64